In [1]:
%pip install -qU langchain
%pip install -qU langchain-openai
%pip install -qU langchain-groq
%pip install -qU langchain-qdrant



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_API_URL = os.getenv("QDRANT_URL")

# LangSmith tracing and 
os.environ["LANGCHAIN_PROJECT"] = "AirBnB PDF Jun18"
os.environ["LANGCHAIN_ENDPOINT"]=os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv("LANGCHAIN_TRACING_V2")

In [8]:
import os
from langchain import hub
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant

llm = ChatGroq(model="llama3-70b-8192", temperature=0.3)

# Leverage a prompt from the LangChain hub
LLAMA3_PROMPT = hub.pull("cracked-nut/securities-comm-llama3-v2")

# Parameterize some stuff
EMBEDDING_MODEL = OpenAIEmbeddings(model="text-embedding-3-small")
LOAD_NEW_DATA = False
FILE_PATH = "/home/donbr/aie3-bootcamp/AIE3/Week 3/Day 2/files/airbnb.pdf"
COLLECTION_NAME = "airbnb_pdf_rec_1000_200_images"

# QUESTION = "What is Airbnb's 'Description of Business'?"
QUESTION = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
# QUESTION = "What is the 'maximum number of shares to be sold under the 10b5-1 Trading plan' by Brian Chesky?"

In [9]:
qdrant = Qdrant.from_existing_collection(
    embedding=EMBEDDING_MODEL,
    collection_name=COLLECTION_NAME,
    url=QDRANT_API_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,   
)

retriever = qdrant.as_retriever(search_kwargs={"k": 5})


In [10]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": LLAMA3_PROMPT | llm, "context": itemgetter("context")}
)

In [11]:
response = retrieval_augmented_qa_chain.invoke({"question" : QUESTION})

In [12]:
# return the response.  filter on the response key AIMessage content element
print(response["response"].content)


The total value of 'Cash and cash equivalents' as of December 31, 2023, is $2,369. This information can be found in the following excerpt from the document:

"Cash and cash equivalents:
Money market funds
$2,018 
Certificates of deposit
— 
1 
— 
1 
Government bonds
— 
115 
— 
115 
Commercial paper
— 
223 
— 
223 
Corporate debt securities
— 
12 
— 
12 
$2,018 
351 
— 
$2,369"

This excerpt shows the breakdown of cash and cash equivalents as of December 31, 2023, with a total value of $2,369.


In [13]:
response["context"]

[Document(page_content='December 31, 2023\nLevel\xa01\nLevel\xa02\nLevel\xa03\nTotal\nAssets\nCash and cash equivalents:\nMoney market funds\n$\n2,018\xa0 $\n—\xa0 $\n—\xa0 $\n2,018\xa0\nCertificates of deposit\n—\xa0\n1\xa0\n—\xa0\n1\xa0\nGovernment bonds\n—\xa0\n115\xa0\n—\xa0\n115\xa0\nCommercial paper\n—\xa0\n223\xa0\n—\xa0\n223\xa0\nCorporate debt securities\n—\xa0\n12\xa0\n—\xa0\n12\xa0\n2,018\xa0\n351\xa0\n—\xa0\n2,369\xa0\nShort-term investments:\nCertificates of deposit\n—\xa0\n172\xa0\n—\xa0\n172\xa0\nGovernment bonds\n—\xa0\n333\xa0\n—\xa0\n333\xa0\nCommercial paper\n—\xa0\n366\xa0\n—\xa0\n366\xa0\nCorporate debt securities\n—\xa0\n1,491\xa0\n—\xa0\n1,491\xa0\nMortgage-backed and asset-backed securities\n—\xa0\n145\xa0\n—\xa0\n145\xa0\n—\xa0\n2,507\xa0\n—\xa0\n2,507\xa0\nFunds receivable and amounts held on behalf of customers:\nMoney market funds\n1,360\xa0\n—\xa0\n—\xa0\n1,360\xa0\nPrepaids and other current assets:\nForeign exchange derivative assets\n—\xa0\n27\xa0\n—\xa0